# 🔍 Unobserved Components Model with State Space & Kalman Filtering

This notebook demonstrates how to build and estimate an unobserved components model using Econometron's state space utilities, simulated annealing for parameter estimation, and the Kalman filter/smoother. We use the first row of `Z.mat` as the observed series.

## 1️⃣ Introduction

Unobserved components models decompose a time series into latent (hidden) components such as trend and cycle. State space methods and the Kalman filter are ideal for estimation and inference in such models.

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from econometron.utils.state_space.update_ss import make_state_space_updater
from econometron.utils.optimizers.simulated_annealing import simulated_annealing
from econometron.utils.filters.kalman import kalman_filter, kalman_smoother

## 2️⃣ Load Data

We use the first row of `Z.mat` (e.g., interest rate series).

In [ ]:
# Load data
Z = scipy.io.loadmat('Z.mat')["Z"]
y = Z[0, :]
plt.plot(y)
plt.title('Observed Series (row 0 of Z)')
plt.show()

## 3️⃣ Define State Space Model

We use a local level model as an example:
$$
y_t = \\mu_t + \\epsilon_t \\
\\mu_{t+1} = \\mu_t + \\eta_t
$$
where $\\epsilon_t \sim N(0, R)$ and $\\eta_t \sim N(0, Q)$.

The solver returns matrices $A$ and $B$ for the state transition and observation equations.

In [ ]:
# State space builder and solver
def solver(params):
    # params: [Q, R]
    Q, R = params
    # State transition: mu_{t+1} = mu_t + eta_t
    A = np.array([[1]])
    B = np.array([[1]])
    return A, B, Q, R

def build_R(params):
    return np.array([[params[1]]])

def build_C(params):
    return np.array([[params[0]]])

# Initial parameters
base_params = {'Q': 1.0, 'R': 1.0}

# State space updater
ss_updater = make_state_space_updater(
    base_params=base_params,
    solver=solver,
    build_R=build_R,
    build_C=build_C
)

## 4️⃣ Estimate Parameters (Simulated Annealing + Kalman Filter)

We estimate Q and R by maximizing the likelihood via simulated annealing.

In [ ]:
def neg_log_likelihood(params):
    A, B, Q, R = solver(params)
    # Build state space matrices
    n = len(y)
    x0 = np.array([y[0]])
    P0 = np.eye(1)
    Qm = np.array([[Q]])
    Rm = np.array([[R]])
    _, ll = kalman_filter(y, A, B, Qm, Rm, x0, P0, return_loglik=True)
    return -ll

# Simulated annealing to estimate Q, R
bounds = [(1e-5, 10.0), (1e-5, 10.0)]
best_params, best_ll = simulated_annealing(neg_log_likelihood, x0=[1.0, 1.0], bounds=bounds, maxiter=100)
print('Estimated Q, R:', best_params)

## 5️⃣ Kalman Smoother and Plot

After estimation, use the Kalman smoother to extract the smoothed state (trend) and plot it.

In [ ]:
# Build final state space with estimated params
A, B, Q, R = solver(best_params)
Qm = np.array([[Q]])
Rm = np.array([[R]])
x0 = np.array([y[0]])
P0 = np.eye(1)

# Run Kalman filter and smoother
filtered, _ = kalman_filter(y, A, B, Qm, Rm, x0, P0)
smoothed = kalman_smoother(y, A, B, Qm, Rm, x0, P0)

# Plot
plt.figure(figsize=(12, 5))
plt.plot(y, label='Observed')
plt.plot(smoothed[:, 0], label='Smoothed Trend', linewidth=2)
plt.title('Unobserved Components Model: Smoothed Trend')
plt.legend()
plt.show()

## 6️⃣ Conclusion

- We built an unobserved components model using Econometron's state space utilities.
- Parameters were estimated using simulated annealing and the Kalman filter.
- The Kalman smoother provided a clean estimate of the underlying trend.

This workflow can be extended to more complex state space models and real-world macroeconomic data.